In [1]:
import Serializer as srl
import socket
import tensorforce
from tensorforce import Agent, Environment

In [2]:
class PongEnvironment(Environment):
    def __init__(self):
        super().__init__()

    def states(self):
        return dict(type='float', shape=(4,))

    def actions(self):
        return dict(type='int', num_values=3)

    # Optional: should only be defined if environment has a natural fixed
    # maximum episode length; restrict training timesteps via
    #     Environment.create(..., max_episode_timesteps=???)
    # def max_episode_timesteps(self):
    #     return super().max_episode_timesteps()

    # Optional additional steps to close environment
    # def close(self):
    #     super().close()

    def reset(self):
        # state = np.random.random(size=(8,))
        # return state
        pass

    def execute(self, actions):
        # next_state = np.random.random(size=(8,))
        # terminal = np.random.random() < 0.5
        # reward = np.random.random()
        # return next_state, terminal, reward
        pass

In [3]:
def CreateAgent():
    return Agent.create(
    agent='ppo',
    # environment=PongEnvironment(),  # alternatively: states, actions, (max_episode_timesteps),
    states=dict(type='float', shape=(6,)),
    actions=dict(type='int', num_values=3),
    max_episode_timesteps=1000,
    network='auto',
    # PPO optimization parameters
    batch_size=10, update_frequency=2, learning_rate=3e-4,
    subsampling_fraction=0.33,
    # Reward estimation
    likelihood_ratio_clipping=0.2, discount=0.99,
    # Preprocessing
    preprocessing='linear_normalization',
    # Exploration
    exploration=0.0, variable_noise=0.0,
    # Regularization
    l2_regularization=0.0, entropy_regularization=0.0,
    # Default additional config values
    config=None,
    # Log all available Tensorboard summaries
    summarizer=dict(directory='summaries', labels='all'),
    # Do not record agent-environment interaction trace
    recorder=None
)

In [4]:
def Act(agent, conn):
    obs = srl.DeserializeMessage(conn)
#     print('obs:', obs)
    act = agent.act(obs)
#     print('message:', srl.SerializeInt(act))
    conn.sendall(srl.SerializeInt(int(act) - 1).encode(FORMAT))
#     print('acted:', act)
    return act

    
def Learn(agent, conn):
    reward, done = srl.DeserializeMessage(conn), bool(srl.DeserializeMessage(conn))
    agent.observe(terminal=done, reward=reward)
    return reward, done

In [5]:
AgentL, AgentR = CreateAgent(), CreateAgent()
print('ok')

ok


In [ ]:
HOST = '127.0.0.1'
PORTL = 1488
PORTR = 1489
FORMAT = 'utf-8'

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORTL))
    print('waiting for L to connect')
    s.listen()
    connL, addrL = s.accept()
    print('L Connected by', addrL)


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORTR))
    s.listen()
    print('waiting for R to connect')
    connR, addrR = s.accept()
    print('R Connected by', addrR)

EPOCH_NUM = 200
for epoch in range(EPOCH_NUM):
    print(f'epoch: {epoch: 4}')

    doneL, doneR = False, False
    rewardTotalL, rewardTotalR = 0, 0
    while not (doneL or doneR):
#         print('____________________')
        actL = Act(AgentL, connL)
#         print(f'actL {actL}')
        actR = Act(AgentR, connR)
#         print(f'actR {actR}')
#         print('it ended')

        rewardL, doneL = Learn(AgentL, connL)
        rewardR, doneR = Learn(AgentR, connR)
        rewardTotalL += rewardL
        rewardTotalR += rewardR
        
    print(f'rewards: {rewardTotalL} {rewardTotalR}')
#         print(f'done {doneL} {doneR}')

connL.close()
connR.close()

waiting for L to connect
L Connected by ('127.0.0.1', 51207)
waiting for R to connect
R Connected by ('127.0.0.1', 51208)
epoch:    0
rewards: -99976 100024
epoch:    1
rewards: 100069 -99931
epoch:    2
rewards: 100065 -99935
epoch:    3
rewards: 100044 -99956
epoch:    4
rewards: -99980 100020
epoch:    5
rewards: -99957 100043
epoch:    6
rewards: -99980 100020
epoch:    7
rewards: -99954 100046
epoch:    8
rewards: -99979 100021
epoch:    9
rewards: 100046 -99954
epoch:   10
rewards: 100021 -99979
epoch:   11
rewards: -99977 100023
epoch:   12
rewards: 100066 -99934
epoch:   13
rewards: -99952 100048
epoch:   14
rewards: 100065 -99935
epoch:   15
rewards: -99952 100048
epoch:   16
rewards: 100021 -99979
epoch:   17
rewards: 100022 -99978
epoch:   18
rewards: 100022 -99978
epoch:   19
rewards: 100022 -99978
epoch:   20
rewards: -99978 100022
epoch:   21
rewards: -99950 100050
epoch:   22
rewards: -99952 100048
epoch:   23
rewards: -99977 100023
epoch:   24
rewards: -99978 100022
epo